# Wandb Results downloader

This notebook allows to download as CSV file the results got in Wandb

In [2]:
import numpy as np
import pandas as pd
import wandb
from ezdl.utils.utilities import mlflow_linearize

/home/pasquale/projects/LWViTs-for-weedmapping/venv/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
api = wandb.Api()

In [4]:
from IPython.core.display_functions import display


def get_runs(run_filters, project="pasqualedem/weeddetection"):
    # Project is specified by <entity/project-name>
    runs = api.runs(project, filters=run_filters)

    summary_list, config_list, name_list = [], [], []
    for run in runs:
        # .summary contains the output keys/values for metrics like accuracy.
        #  We call ._json_dict to omit large files
        summary_list.append(run.summary._json_dict)

        # .config contains the hyperparameters.
        #  We remove special values that start with _.
        config_list.append(
            {k: v for k,v in run.config.items()
              if not k.startswith('_')})

        # .name is the human-readable name of the run.
        name_list.append(run.name)
    df = pd.DataFrame([{**mlflow_linearize(dc), **mlflow_linearize(ds)} for dc, ds in zip(config_list, summary_list)])
    return df

def remove_hyp(df):
    to_delete = filter(lambda x: x.startswith('hyper_params'), df.columns)
    for l in to_delete:
        df.drop(l, inplace=True)

def rename_values(df: pd.DataFrame, vrenamings: dict):
    for column, values in vrenamings.items():
        if column in df.columns:
            df[column] = df[column].replace(values)

In [59]:
renamings = {
    'in_params.dataset.channels': 'Channels',
    'in_params.train_params.optimizer_params.weight_decay': "Weight Decay",
    "in_params.model.params.side_pretrained": "Used Weights",
    'in_params.train_params.initial_lr': 'LR',
    'in_params.model.name': 'Model',
    'in_params.model.params.backbone': 'Backbone',
    'in_params.train_params.freeze_pretrained': 'Strategy',
    'in_params.model.params.main_pretrained': 'Used Weights',
    'train_WrapF1': 'Train F1',
    'f1': 'F1',
    'auc': 'AUC',
    'in_params.dataset.batch_size': 'Batch Size',
    'precision': 'Precision',
    "in_params.train_params.loss.name": 'Loss',
    'in_params.train_params.loss.params.gamma': 'Gamma',
    'in_params.model.params.fusion_type': 'Fusion',
    'recall': 'Recall'
}
value_renamings = {
    'Strategy': {True: 'Freeze', False: 'Fine-Tuning'},
    'Fusion': {'conv_sum_drop': 'Conv Drop', 'conv_sum': 'Conv', "squeeze_excite": "SE"},
    'Model': {"wd/models/splitlawin": "SplitLawin", "splitlawin": "SplitLawin", 'doublelawin': 'DoubleLawin'},
    'Loss': {'cross_entropy': 'Cross Entropy', 'focal': 'Focal'},
    'Used Weights': {0: 'False', 'G': True },
}
channels_ordering = {
    "G, NDVI, NIR, R, RE" : 1,
    "R, G, NIR, RE": 2,
    "R, G, RE, NIR": 2,
    "R, G, RE": 3,
    "G, NIR, R": 3,
    "R, G, NIR": 4,
    "R, G, NDVI": 5,
    "R, G": 6,
    "CIR": 7,
    "NDVI": 8,

    'Fine-Tuning': 1,
    'Freeze': 2,

    'R': 1,
    'G': 2,
    'RGB': 2.5,
    'G, G, R': 3,
    'R, G, G, G': 4,

    'MiT-B0': 0,
    'MiT-B1': 1,
    'lawin': 0,
    'laweed': 1,
    'doublelawin': 2,
    'doublelaweed': 3,
}

# Sequoia

## Final Results

In [24]:
filters = {
    "$or":[
        {"$and": [
            {'$or': [{'config.in_params.model.name': 'splitlawin'},
                    {'config.in_params.model.name': 'doublelawin'},
                    ]},
            {'config.in_params.train_params.loss.name': 'focal'}
        ]},
        {"$or": [
            {'config.in_params.model.name': 'lawin'},
            {'config.in_params.model.name': 'segnet'},
            ] }
    ]    
}
selected_cols = [
    'in_params.model.name',
    'in_params.model.params.backbone',
    'in_params.model.params.fusion_type',
    'in_params.train_params.freeze_pretrained',
    'in_params.model.params.main_pretrained',
    'in_params.model.params.side_pretrained',
    'in_params.dataset.channels',
    'f1',
    'auc',
    'auc_background',
    'auc_weed',
    'auc_crop',
]
roundings = ['F1', 'AUC']
runs_df = get_runs(filters, project="weedmapping-sequoia")

In [25]:
len(runs_df)

211

In [26]:
projected_runs = runs_df[
    selected_cols
]
selected_runs = projected_runs

In [27]:
selected_runs.rename(columns=renamings, inplace=True)
rename_values(selected_runs, value_renamings)
selected_runs[roundings] = selected_runs[roundings].round(5)
selected_runs.fillna(0, inplace=True)
selected_runs = selected_runs.applymap(lambda x: ', '.join(x) if isinstance(x, list) else x)
selected_runs.sort_values(by=['Model', 'Backbone'], inplace=True)
idx = selected_runs.groupby(by=['Model', 'Backbone'])['F1'].transform(max) == selected_runs['F1']
best_runs = selected_runs[idx]
best_runs[roundings] = best_runs[roundings].round(3)
selected_runs[roundings] = selected_runs[roundings].round(3)

/tmp/ipykernel_465561/578636069.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_runs.rename(columns=renamings, inplace=True)
/tmp/ipykernel_465561/2946659532.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column] = df[column].replace(values)
/tmp/ipykernel_465561/578636069.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-

In [28]:
best_runs

,Model,Backbone,Fusion,Strategy,Used Weights,Multispectral Pretraining,Channels,F1,AUC,auc_background,auc_weed,auc_crop
172,DoubleLawin,MiT-B0,Conv,Freeze,0,True,"R, G, NDVI",0.601,0.973,0.996500,0.936209,0.985029
118,DoubleLawin,MiT-B1,Conv Drop,Freeze,0,True,"R, G, NDVI",0.580,0.987,0.994749,0.992764,0.974159
109,SplitLawin,MiT-B0,Conv Drop,Fine-Tuning,0,True,"R, G, NDVI",0.635,0.970,0.990135,0.935838,0.983149
101,SplitLawin,MiT-B1,Conv Drop,Fine-Tuning,0,True,"R, G, NDVI",0.630,0.976,0.997336,0.941123,0.988723
1,lawin,MiT-B0,0,Fine-Tuning,"R, G",0,"R, G, NIR, RE",0.663,0.000,0.991476,0.974586,0.966516
184,lawin,MiT-B1,0,Freeze,0,0,NDVI,0.685,0.970,0.979865,0.988934,0.942581
210,segnet,0,0,0,0,0,NDVI,0.445,0.974,0.987260,0.951266,0.984912


In [ ]:
filters = {
}
selected_cols = [
    'in_params.dataset.channels',
    'in_params.model.name',
    'in_params.model.params.backbone',
    'f1',
]
roundings = ['F1']
runs_df = get_runs(filters)
remove_hyp(runs_df)

projected_runs = runs_df[
    selected_cols
]
selected_runs = projected_runs

In [ ]:
selected_runs.rename(columns=renamings, inplace=True)
rename_values(selected_runs, value_renamings)
selected_runs[roundings] = selected_runs[roundings].round(5)
selected_runs.fillna(0, inplace=True)
selected_runs = selected_runs.applymap(lambda x: ', '.join(x) if isinstance(x, list) else x)
idx = selected_runs.groupby(by=['Channels'])['F1'].transform(max) == selected_runs['F1']
best_runs_channel = selected_runs[idx]
best_runs_channel[roundings] = best_runs_channel[roundings].round(3)
selected_runs[roundings] = selected_runs[roundings].round(3)
best_runs_channel

## Pretrained Split-Double Sequoia

In [67]:
filters = {
    "$and": [
    {"$or":[
        {'config.in_params.model.name': 'splitlawin'},
        {'config.in_params.model.name': 'doublelawin'}, 
    ]},
    {'config.in_params.train_params.loss.name': 'focal'}
    ]
}
selected_cols = [
    'in_params.model.name',
    'in_params.model.params.fusion_type',
    "in_params.model.params.side_pretrained",
    'in_params.model.params.backbone',
    'f1',
]
roundings = ['F1']
runs_df = get_runs(filters, project="pasqualedem/weedmapping-sequoia")

In [68]:
projected_runs = runs_df[
    selected_cols
]
selected_runs = projected_runs

In [75]:
selected_runs.rename(columns=renamings, inplace=True)
rename_values(selected_runs, value_renamings)
selected_runs[roundings] = selected_runs[roundings].round(5)
selected_runs.fillna(0, inplace=True)
selected_runs = selected_runs.applymap(lambda x: ', '.join(x) if isinstance(x, list) else x)
selected_runs.sort_values(by=['Multispectral Pretraining', 'Model', 'Backbone'], inplace=True)
idx = selected_runs.groupby(by=['Model', 'Backbone', 'Multispectral Pretraining'])['F1'].transform(max) == selected_runs['F1']
best_runs = selected_runs[idx]
best_runs[roundings] = best_runs[roundings].round(3)
selected_runs[roundings] = selected_runs[roundings].round(3)

/tmp/ipykernel_441543/1660266559.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  best_runs[roundings] = best_runs[roundings].round(3)


In [76]:
best_runs

,Model,Fusion,Multispectral Pretraining,Backbone,F1
142,DoubleLawin,Conv,True,MiT-B0,0.601
114,DoubleLawin,Conv Drop,True,MiT-B1,0.580
105,SplitLawin,Conv Drop,True,MiT-B0,0.635
97,SplitLawin,Conv Drop,True,MiT-B1,0.630
59,DoubleLawin,SE,False,MiT-B0,0.519
52,DoubleLawin,Conv Drop,False,MiT-B1,0.541
86,SplitLawin,SE,False,MiT-B0,0.582
80,SplitLawin,Conv,False,MiT-B1,0.571


## Pretrained Lawin

In [98]:
filters = {
    "$and": [
    {'config.in_params.model.name': 'lawin'}, 
    {'config.in_params.train_params.loss.name': 'focal'}
    ]
}
selected_cols = [
    'in_params.model.name',
    "in_params.model.params.main_pretrained",
    'in_params.model.params.backbone',
    'in_params.dataset.channels',
    'f1',
]
roundings = ['F1']
runs_df = get_runs(filters, project="pasqualedem/weedmapping-sequoia")

In [99]:
projected_runs = runs_df[
    selected_cols
]
selected_runs = projected_runs

In [100]:
selected_runs.rename(columns=renamings, inplace=True)
rename_values(selected_runs, value_renamings)
selected_runs[roundings] = selected_runs[roundings].round(5)
selected_runs.fillna(0, inplace=True)
selected_runs = selected_runs.applymap(lambda x: ', '.join(x) if isinstance(x, list) else x)
selected_runs.sort_values(by=['Used Weights', 'Model', 'Backbone'], inplace=True)
idx = selected_runs.groupby(by=['Model', 'Backbone', 'Used Weights'])['F1'].transform(max) == selected_runs['F1']
best_runs = selected_runs[idx]
best_runs[roundings] = best_runs[roundings].round(3)
selected_runs[roundings] = selected_runs[roundings].round(3)

/tmp/ipykernel_441543/681276384.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_runs.rename(columns=renamings, inplace=True)
/tmp/ipykernel_441543/2946659532.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column] = df[column].replace(values)
/tmp/ipykernel_441543/681276384.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-

In [101]:
best_runs

,Model,Used Weights,Backbone,Channels,F1
26,lawin,0,MiT-B0,NDVI,0.645
32,lawin,0,MiT-B1,NDVI,0.685
17,lawin,G,MiT-B0,NDVI,0.646
15,lawin,G,MiT-B1,NDVI,0.656
16,lawin,"G, G, R",MiT-B0,CIR,0.527
12,lawin,"G, G, R",MiT-B1,CIR,0.582
1,lawin,"R, G",MiT-B0,"R, G, NIR, RE",0.663
0,lawin,"R, G",MiT-B1,"R, G, NIR, RE",0.610
21,lawin,"R, G, G, G",MiT-B0,"R, G, NIR, RE",0.499
11,lawin,"R, G, G, G",MiT-B1,"R, G, NIR, RE",0.493


## SplitLawin Sequoia

In [94]:
filters = {
    "$and": [
    {'config.in_params.model.name': 'splitlawin'},
    {'config.in_params.train_params.loss.name': 'focal'}
    ]
}
selected_cols = [
    'in_params.model.name',
    'in_params.model.params.backbone',
    'in_params.train_params.freeze_pretrained',
    'in_params.model.params.fusion_type',
    "in_params.model.params.side_pretrained",
    'in_params.dataset.channels',
    'f1',
    'auc',
    'auc_background',
    'auc_weed',
    'auc_crop',
]
roundings = ['F1', 'AUC']
runs_df = get_runs(filters, project="pasqualedem/weedmapping-sequoia")

In [95]:
len(runs_df)

72

In [96]:
projected_runs = runs_df[
    selected_cols
]
selected_runs = projected_runs

In [97]:
selected_runs.rename(columns=renamings, inplace=True)
selected_runs.fillna(0, inplace=True)
rename_values(selected_runs, value_renamings)
selected_runs[roundings] = selected_runs[roundings].round(5)
selected_runs['AUC'] = (selected_runs['auc_background'] + selected_runs['auc_weed'] + selected_runs['auc_crop']) / 3
selected_runs.drop(columns=['auc_background', 'auc_weed', 'auc_crop'], inplace=True)   
selected_runs['Used Weights'] = selected_runs.apply(lambda x: ['R', 'G', 'B'] if x['Used Weights'] == 'False' else ['R', 'G'] + ['G'] * (len(x['Channels']) - 2), axis=1)
selected_runs = selected_runs.applymap(lambda x: ', '.join(x) if isinstance(x, list) else x)
selected_runs.sort_values(by=['Backbone', 'Strategy', 'Fusion', 'Used Weights',  'Channels'], inplace=True)
selected_runs[roundings] = selected_runs[roundings].round(3)

/tmp/ipykernel_465561/1074957724.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_runs.rename(columns=renamings, inplace=True)
/tmp/ipykernel_465561/1074957724.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_runs.fillna(0, inplace=True)
/tmp/ipykernel_465561/2946659532.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column] = df[column].replace(va

In [98]:
selected_runs

,Model,Backbone,Strategy,Fusion,Used Weights,Channels,F1,AUC
53,SplitLawin,MiT-B0,Fine-Tuning,Conv,"R, G, G","R, G, NDVI",0.349,0.893
54,SplitLawin,MiT-B0,Fine-Tuning,Conv,"R, G, G","R, G, NIR",0.378,0.900
52,SplitLawin,MiT-B0,Fine-Tuning,Conv,"R, G, G","R, G, RE",0.425,0.943
55,SplitLawin,MiT-B0,Fine-Tuning,Conv,"R, G, G, G","R, G, NIR, RE",0.406,0.951
49,SplitLawin,MiT-B0,Fine-Tuning,Conv Drop,"R, G, G","R, G, NDVI",0.635,0.970
...,...,...,...,...,...,...,...,...
16,SplitLawin,MiT-B1,Freeze,SE,"R, G, B","R, G, RE",0.560,0.970
9,SplitLawin,MiT-B1,Freeze,SE,"R, G, G","R, G, NDVI",0.422,0.988
10,SplitLawin,MiT-B1,Freeze,SE,"R, G, G","R, G, NIR",0.550,0.956
8,SplitLawin,MiT-B1,Freeze,SE,"R, G, G","R, G, RE",0.584,0.968


In [99]:
selected_runs.drop(columns=['Model']).to_csv("results/SplitLawin_Sequoia.csv", index=False)

## DoubleLawin Sequoia

In [116]:
filters = {
    "$and": [
    {'config.in_params.model.name': 'doublelawin'},
    {'config.in_params.train_params.loss.name': 'focal'}
    ]
}
selected_cols = [
    'in_params.model.name',
    'in_params.model.params.backbone',
    'in_params.model.params.fusion_type',
    "in_params.model.params.side_pretrained",
    'in_params.train_params.freeze_pretrained',
    'in_params.dataset.channels',
    'f1',
    'auc',
    'auc_background',
    'auc_weed',
    'auc_crop',
]
roundings = ['F1', 'AUC']
runs_df = get_runs(filters, project="pasqualedem/weedmapping-sequoia")

In [117]:
len(runs_df)

72

In [118]:
projected_runs = runs_df[
    selected_cols
]
selected_runs = projected_runs

In [119]:
selected_runs.rename(columns=renamings, inplace=True)
selected_runs.fillna(0, inplace=True)
rename_values(selected_runs, value_renamings)
selected_runs[roundings] = selected_runs[roundings].round(5)
selected_runs['AUC'] = (selected_runs['auc_background'] + selected_runs['auc_weed'] + selected_runs['auc_crop']) / 3
selected_runs.drop(columns=['auc_background', 'auc_weed', 'auc_crop'], inplace=True)   
selected_runs['Used Weights'] = selected_runs.apply(lambda x: ['R', 'G', 'B'] if x['Used Weights'] == 'False' else ['R', 'G'] + ['G'] * (len(x['Channels']) - 2), axis=1)
selected_runs = selected_runs.applymap(lambda x: ', '.join(x) if isinstance(x, list) else x)
selected_runs.sort_values(by=['Backbone', 'Strategy', 'Fusion', 'Used Weights',  'Channels'], inplace=True)
selected_runs[roundings] = selected_runs[roundings].round(3)

/tmp/ipykernel_465561/1074957724.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_runs.rename(columns=renamings, inplace=True)
/tmp/ipykernel_465561/1074957724.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_runs.fillna(0, inplace=True)
/tmp/ipykernel_465561/2946659532.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column] = df[column].replace(va

In [120]:
selected_runs

,Model,Backbone,Fusion,Used Weights,Strategy,Channels,F1,AUC
30,DoubleLawin,MiT-B0,Conv,"R, G, G",Fine-Tuning,"R, G, NDVI",0.373,0.975
31,DoubleLawin,MiT-B0,Conv,"R, G, G",Fine-Tuning,"R, G, NIR",0.450,0.945
29,DoubleLawin,MiT-B0,Conv,"R, G, G",Fine-Tuning,"R, G, RE",0.476,0.936
28,DoubleLawin,MiT-B0,Conv,"R, G, G, G",Fine-Tuning,"R, G, NIR, RE",0.542,0.961
26,DoubleLawin,MiT-B0,Conv Drop,"R, G, G",Fine-Tuning,"R, G, NDVI",0.421,0.990
...,...,...,...,...,...,...,...,...
33,DoubleLawin,MiT-B1,SE,"R, G, B",Freeze,"R, G, RE",0.520,0.935
2,DoubleLawin,MiT-B1,SE,"R, G, G",Freeze,"R, G, NDVI",0.540,0.980
3,DoubleLawin,MiT-B1,SE,"R, G, G",Freeze,"R, G, NIR",0.455,0.954
1,DoubleLawin,MiT-B1,SE,"R, G, G",Freeze,"R, G, RE",0.472,0.949


In [121]:
selected_runs.drop(columns=['Model']).to_csv("results/DoubleLawin_Sequoia.csv", index=False)

## Lawin Sequoia

In [19]:
filters = {
    "$and": [
    {'config.in_params.model.name': 'lawin'}, 
    {'config.in_params.train_params.loss.name': 'focal'}
    ]
}
selected_cols = [
    'in_params.model.name',
    'in_params.model.params.backbone',
    'in_params.train_params.freeze_pretrained',
    "in_params.model.params.main_pretrained",
    'in_params.dataset.channels',
    'f1',
    'auc',
    'auc_background',
    'auc_weed',
    'auc_crop',
]
roundings = ['F1', 'AUC']
runs_df = get_runs(filters, project="pasqualedem/weedmapping-sequoia")

In [20]:
len(runs_df)

26

In [22]:
projected_runs = runs_df[
    selected_cols
]
selected_runs = projected_runs

selected_runs.rename(columns=renamings, inplace=True)
selected_runs.fillna(0, inplace=True)
rename_values(selected_runs, value_renamings)
selected_runs[roundings] = selected_runs[roundings].round(5)
selected_runs['AUC'] = (selected_runs['auc_background'] + selected_runs['auc_weed'] + selected_runs['auc_crop']) / 3
selected_runs['Used Weights'] = selected_runs.apply(lambda x: x['Used Weights'] if x['Used Weights'] != 'G' else ['G'] * len(x['Channels']), axis=1)
selected_runs['Used Weights'] = selected_runs.apply(lambda x: x['Used Weights'] if x['Used Weights'] != 0 else ['R', 'G', 'B'][:len(x['Channels'])], axis=1)
selected_runs.drop(columns=['auc_background', 'auc_weed', 'auc_crop'], inplace=True)   
selected_runs = selected_runs.applymap(lambda x: ', '.join(x) if isinstance(x, list) else x)
selected_runs.sort_values(by=['Backbone', 'Strategy', 'Used Weights', 'Channels'], inplace=True)
selected_runs[roundings] = selected_runs[roundings].round(3)

/tmp/ipykernel_465561/412777604.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_runs.rename(columns=renamings, inplace=True)
/tmp/ipykernel_465561/412777604.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_runs.fillna(0, inplace=True)
/tmp/ipykernel_465561/2946659532.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column] = df[column].replace(valu

In [23]:
selected_runs.drop(columns=['Model']).to_csv("results/Lawin_Sequoia.csv", index=False)

# RedEdge

## Final Results

In [34]:
filters = {
    "$or":[
        {"$and": [
            {'$or': [{'config.in_params.model.name': 'splitlawin'},
                    {'config.in_params.model.name': 'doublelawin'},
                    ]},
            {'config.in_params.train_params.loss.name': 'focal'}
        ]},
        {"$or": [
            {'config.in_params.model.name': 'lawin'},
            {'config.in_params.model.name': 'segnet'},
            ] }
    ]    
}
selected_cols = [
    'in_params.model.name',
    'in_params.model.params.backbone',
    'in_params.train_params.freeze_pretrained',
    "in_params.model.params.main_pretrained",
    'in_params.model.params.side_pretrained',
    'in_params.dataset.channels',
    'in_params.model.params.fusion_type',
    'f1',
    'auc',
    'auc_background',
    'auc_weed',
    'auc_crop',
]
roundings = ['F1', 'AUC']
runs_df = get_runs(filters, project="weedmapping-rededge")

In [35]:
len(runs_df)

222

In [36]:
projected_runs = runs_df[
    selected_cols
]
selected_runs = projected_runs

In [37]:
selected_runs.rename(columns=renamings, inplace=True)
rename_values(selected_runs, value_renamings)
selected_runs[roundings] = selected_runs[roundings].round(5)
selected_runs.fillna(0, inplace=True)
selected_runs = selected_runs.applymap(lambda x: ', '.join(x) if isinstance(x, list) else x)
selected_runs.sort_values(by=['Model', 'Backbone'], inplace=True)
idx = selected_runs.groupby(by=['Model', 'Backbone'])['F1'].transform(max) == selected_runs['F1']
best_runs = selected_runs[idx]
best_runs[roundings] = best_runs[roundings].round(3)
selected_runs[roundings] = selected_runs[roundings].round(3)

/tmp/ipykernel_465561/578636069.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_runs.rename(columns=renamings, inplace=True)
/tmp/ipykernel_465561/2946659532.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column] = df[column].replace(values)
/tmp/ipykernel_465561/578636069.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-

In [38]:
best_runs

,Model,Backbone,Strategy,Used Weights,Multispectral Pretraining,Channels,Fusion,F1,AUC,auc_background,auc_weed,auc_crop
35,DoubleLawin,MiT-B0,Fine-Tuning,0,True,"R, G, B, NDVI",Conv,0.863,0.000,0.963957,0.937385,0.991533
24,DoubleLawin,MiT-B1,Fine-Tuning,0,True,"R, G, B, NIR",Conv Drop,0.863,0.000,0.983083,0.886343,0.989527
14,SplitLawin,MiT-B0,Fine-Tuning,0,0,"R, G, B, NIR, RE",SE,0.864,0.000,0.972517,0.938383,0.960614
9,SplitLawin,MiT-B1,Fine-Tuning,0,0,"R, G, B, NIR, RE",SE,0.865,0.000,0.982775,0.817392,0.985158
170,lawin,MiT-B0,Fine-Tuning,"G, G, R",0,CIR,0,0.856,0.908,0.986902,0.747750,0.988166
169,lawin,MiT-B1,Fine-Tuning,"G, G, R",0,CIR,0,0.857,0.929,0.989439,0.862950,0.935459
135,segnet,0,0,0,0,"B, NIR, G, R, G, NDVI, NIR, R, RE, R, G, B",0,0.854,0.988,0.990339,0.979122,0.994607


## Pretrained SplitDouble

In [77]:
filters = {
    "$and": [
    {"$or":[
        {'config.in_params.model.name': 'splitlawin'},
        {'config.in_params.model.name': 'doublelawin'}, 
    ]},
    {'config.in_params.train_params.loss.name': 'focal'}
    ]
}
selected_cols = [
    'in_params.model.name',
    'in_params.model.params.fusion_type',
    "in_params.model.params.side_pretrained",
    'in_params.model.params.backbone',
    'f1',
]
roundings = ['F1']
runs_df = get_runs(filters, project="pasqualedem/weedmapping-rededge")

In [78]:
projected_runs = runs_df[
    selected_cols
]
selected_runs = projected_runs

In [79]:
selected_runs.rename(columns=renamings, inplace=True)
rename_values(selected_runs, value_renamings)
selected_runs[roundings] = selected_runs[roundings].round(5)
selected_runs.fillna(0, inplace=True)
selected_runs = selected_runs.applymap(lambda x: ', '.join(x) if isinstance(x, list) else x)
selected_runs.sort_values(by=['Multispectral Pretraining', 'Model', 'Backbone'], inplace=True)
idx = selected_runs.groupby(by=['Model', 'Backbone', 'Multispectral Pretraining'])['F1'].transform(max) == selected_runs['F1']
best_runs = selected_runs[idx]
best_runs[roundings] = best_runs[roundings].round(3)
selected_runs[roundings] = selected_runs[roundings].round(3)

/tmp/ipykernel_441543/1660266559.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_runs.rename(columns=renamings, inplace=True)
/tmp/ipykernel_441543/2946659532.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column] = df[column].replace(values)
/tmp/ipykernel_441543/1660266559.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returnin

In [80]:
best_runs

,Model,Fusion,Multispectral Pretraining,Backbone,F1
50,DoubleLawin,Conv Drop,0,MiT-B0,0.851
39,DoubleLawin,Conv Drop,0,MiT-B1,0.850
13,SplitLawin,SE,0,MiT-B0,0.864
9,SplitLawin,SE,0,MiT-B1,0.865
31,DoubleLawin,Conv,True,MiT-B0,0.863
20,DoubleLawin,Conv Drop,True,MiT-B1,0.863
108,SplitLawin,Conv Drop,True,MiT-B0,0.862
101,SplitLawin,Conv,True,MiT-B1,0.865


## Pretrained Lawin

In [94]:
filters = {
    "$and": [
    {'config.in_params.model.name': 'lawin'}, 
    {'config.in_params.train_params.loss.name': 'focal'}
    ]
}
selected_cols = [
    'in_params.model.name',
    "in_params.model.params.main_pretrained",
    'in_params.model.params.backbone',
    'in_params.dataset.channels',
    'f1',
]
roundings = ['F1']
runs_df = get_runs(filters, project="pasqualedem/weedmapping-rededge")

In [95]:
projected_runs = runs_df[
    selected_cols
]
selected_runs = projected_runs

In [96]:
selected_runs.rename(columns=renamings, inplace=True)
rename_values(selected_runs, value_renamings)
selected_runs[roundings] = selected_runs[roundings].round(5)
selected_runs.fillna(0, inplace=True)
selected_runs = selected_runs.applymap(lambda x: ', '.join(x) if isinstance(x, list) else x)
selected_runs.sort_values(by=['Used Weights', 'Model', 'Backbone'], inplace=True)
idx = selected_runs.groupby(by=['Model', 'Backbone', 'Used Weights'])['F1'].transform(max) == selected_runs['F1']
best_runs = selected_runs[idx]
best_runs[roundings] = best_runs[roundings].round(3)
selected_runs[roundings] = selected_runs[roundings].round(3)

/tmp/ipykernel_441543/681276384.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_runs.rename(columns=renamings, inplace=True)
/tmp/ipykernel_441543/2946659532.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column] = df[column].replace(values)
/tmp/ipykernel_441543/681276384.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-

In [97]:
best_runs

,Model,Used Weights,Backbone,Channels,F1
19,lawin,0,MiT-B0,"R, G, B",0.851
14,lawin,0,MiT-B1,CIR,0.852
13,lawin,G,MiT-B0,NDVI,0.749
10,lawin,G,MiT-B1,NDVI,0.773
7,lawin,"G, G, R",MiT-B0,CIR,0.856
6,lawin,"G, G, R",MiT-B1,CIR,0.857
1,lawin,"R, G, B",MiT-B0,"R, G, B, NIR, RE",0.753
0,lawin,"R, G, B",MiT-B1,"R, G, B, NIR, RE",0.767
2,lawin,"R, G, B, G, G",MiT-B0,"R, G, B, NIR, RE",0.856
3,lawin,"R, G, B, G, G",MiT-B1,"R, G, B, NIR, RE",0.855


## Lawin RedEdge

In [14]:
filters = {
    "$and": [
    {'config.in_params.model.name': 'lawin'}, 
    {'config.in_params.train_params.loss.name': 'focal'}
    ]
}
selected_cols = [
    'in_params.model.name',
    'in_params.model.params.backbone',
    'in_params.train_params.freeze_pretrained',
    "in_params.model.params.main_pretrained",
    'in_params.dataset.channels',
    'f1',
    'auc',
    'auc_background',
    'auc_weed',
    'auc_crop',
]
roundings = ['F1', 'AUC']
runs_df = get_runs(filters, project="pasqualedem/weedmapping-rededge")

In [15]:
len(runs_df)

26

In [17]:
projected_runs = runs_df[
    selected_cols
]
selected_runs = projected_runs

selected_runs.rename(columns=renamings, inplace=True)
selected_runs.fillna(0, inplace=True)
rename_values(selected_runs, value_renamings)
selected_runs[roundings] = selected_runs[roundings].round(5)
selected_runs['AUC'] = (selected_runs['auc_background'] + selected_runs['auc_weed'] + selected_runs['auc_crop']) / 3
selected_runs['Used Weights'] = selected_runs.apply(lambda x: x['Used Weights'] if x['Used Weights'] != 'G' else ['G'] * len(x['Channels']), axis=1)
selected_runs['Used Weights'] = selected_runs.apply(lambda x: x['Used Weights'] if x['Used Weights'] != 0 else ['R', 'G', 'B'][:len(x['Channels'])], axis=1)
selected_runs.drop(columns=['auc_background', 'auc_weed', 'auc_crop'], inplace=True)   
selected_runs = selected_runs.applymap(lambda x: ', '.join(x) if isinstance(x, list) else x)
selected_runs.sort_values(by=['Backbone', 'Strategy', 'Used Weights', 'Channels'], inplace=True)
selected_runs[roundings] = selected_runs[roundings].round(3)

/tmp/ipykernel_465561/412777604.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_runs.rename(columns=renamings, inplace=True)
/tmp/ipykernel_465561/412777604.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_runs.fillna(0, inplace=True)
/tmp/ipykernel_465561/2946659532.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column] = df[column].replace(valu

In [18]:
selected_runs.drop(columns=['Model']).to_csv("results/Lawin_RedEdge.csv", index=False)

## DoubleLawin RedEdge

In [106]:
filters = {
    "$and": [
    {'config.in_params.model.name': 'doublelawin'},
    {'config.in_params.train_params.loss.name': 'focal'}
    ]
}
selected_cols = [
    'in_params.model.name',
    'in_params.model.params.backbone',
    'in_params.train_params.freeze_pretrained',
    'in_params.model.params.fusion_type',
    "in_params.model.params.side_pretrained",
    'in_params.dataset.channels',
    'f1',
    'auc',
    'auc_background',
    'auc_weed',
    'auc_crop',
]
roundings = ['F1', 'AUC']
runs_df = get_runs(filters, project="pasqualedem/weedmapping-rededge")

In [107]:
len(runs_df)

72

In [108]:
projected_runs = runs_df[
    selected_cols
]
selected_runs = projected_runs

In [109]:
selected_runs.rename(columns=renamings, inplace=True)
selected_runs.fillna(0, inplace=True)
rename_values(selected_runs, value_renamings)
selected_runs[roundings] = selected_runs[roundings].round(5)
selected_runs['AUC'] = (selected_runs['auc_background'] + selected_runs['auc_weed'] + selected_runs['auc_crop']) / 3
selected_runs.drop(columns=['auc_background', 'auc_weed', 'auc_crop'], inplace=True)   
selected_runs['Used Weights'] = selected_runs.apply(lambda x: ['R', 'G'] if x['Used Weights'] == 'False' else ['R', 'G', 'B'] + ['G'] * (len(x['Channels']) - 3), axis=1)
selected_runs = selected_runs.applymap(lambda x: ', '.join(x) if isinstance(x, list) else x)
selected_runs.sort_values(by=['Backbone', 'Strategy', 'Fusion', 'Used Weights',  'Channels'], inplace=True)
selected_runs[roundings] = selected_runs[roundings].round(3)

/tmp/ipykernel_465561/3061262068.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_runs.rename(columns=renamings, inplace=True)
/tmp/ipykernel_465561/3061262068.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_runs.fillna(0, inplace=True)
/tmp/ipykernel_465561/2946659532.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column] = df[column].replace(va

In [110]:
selected_runs.drop(columns=['Model']).to_csv("results/DoubleLawin_RedEdge.csv", index=False)

## SplitLawin RedEdge

In [111]:
filters = {
    "$and": [
    {'config.in_params.model.name': 'splitlawin'},
    {'config.in_params.train_params.loss.name': 'focal'}
    ]
}
selected_cols = [
    'in_params.model.name',
    'in_params.model.params.backbone',
    'in_params.train_params.freeze_pretrained',
    'in_params.model.params.fusion_type',
    "in_params.model.params.side_pretrained",
    'in_params.dataset.channels',
    'f1',
    'auc',
    'auc_background',
    'auc_weed',
    'auc_crop',
]
roundings = ['F1', 'AUC']
runs_df = get_runs(filters, project="pasqualedem/weedmapping-rededge")

In [112]:
len(runs_df)

72

In [113]:
projected_runs = runs_df[
    selected_cols
]
selected_runs = projected_runs

In [114]:
selected_runs.rename(columns=renamings, inplace=True)
selected_runs.fillna(0, inplace=True)
rename_values(selected_runs, value_renamings)
selected_runs[roundings] = selected_runs[roundings].round(5)
selected_runs['AUC'] = (selected_runs['auc_background'] + selected_runs['auc_weed'] + selected_runs['auc_crop']) / 3
selected_runs.drop(columns=['auc_background', 'auc_weed', 'auc_crop'], inplace=True)   
selected_runs['Used Weights'] = selected_runs.apply(lambda x: ['R', 'G'] if x['Used Weights'] == 'False' else ['R', 'G', 'B'] + ['G'] * (len(x['Channels']) - 3), axis=1)
selected_runs = selected_runs.applymap(lambda x: ', '.join(x) if isinstance(x, list) else x)
selected_runs.sort_values(by=['Backbone', 'Strategy', 'Fusion', 'Used Weights',  'Channels'], inplace=True)
selected_runs[roundings] = selected_runs[roundings].round(3)

/tmp/ipykernel_465561/3061262068.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_runs.rename(columns=renamings, inplace=True)
/tmp/ipykernel_465561/3061262068.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_runs.fillna(0, inplace=True)
/tmp/ipykernel_465561/2946659532.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column] = df[column].replace(va

In [115]:
selected_runs.drop(columns=['Model']).to_csv("results/SplitLawin_RedEdge.csv", index=False)

---
# Fusion blocks

In [60]:
filters = {
    "$and": [
    {"$or":[
        {'config.in_params.model.name': 'splitlawin'},
        {'config.in_params.model.name': 'doublelawin'}, 
    ]},
    {'config.in_params.train_params.loss.name': 'focal'}
    ]
}
selected_cols = [
    'in_params.model.name',
    'in_params.model.params.backbone',
    'in_params.model.params.fusion_type',
    'f1',
]
roundings = ['F1']
runs_df = get_runs(filters, project="pasqualedem/weedmapping-sequoia")

In [61]:
projected_runs = runs_df[
    selected_cols
]
selected_runs = projected_runs

In [62]:
selected_runs.rename(columns=renamings, inplace=True)
rename_values(selected_runs, value_renamings)
selected_runs[roundings] = selected_runs[roundings].round(5)
selected_runs.fillna(0, inplace=True)
selected_runs = selected_runs.applymap(lambda x: ', '.join(x) if isinstance(x, list) else x)
selected_runs.sort_values(by=['Fusion', 'Model', 'Backbone'], inplace=True)
idx = selected_runs.groupby(by=['Model', 'Backbone', 'Fusion'])['F1'].transform(max) == selected_runs['F1']
best_runs = selected_runs[idx]
best_runs[roundings] = best_runs[roundings].round(3)
selected_runs[roundings] = selected_runs[roundings].round(3)

/tmp/ipykernel_441543/2809016558.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_runs.rename(columns=renamings, inplace=True)
/tmp/ipykernel_441543/2946659532.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column] = df[column].replace(values)
/tmp/ipykernel_441543/2809016558.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returnin

In [63]:
best_runs

,Model,Backbone,Fusion,F1
142,DoubleLawin,MiT-B0,Conv,0.601
126,DoubleLawin,MiT-B1,Conv,0.499
94,SplitLawin,MiT-B0,Conv,0.565
80,SplitLawin,MiT-B1,Conv,0.571
138,DoubleLawin,MiT-B0,Conv Drop,0.540
114,DoubleLawin,MiT-B1,Conv Drop,0.580
105,SplitLawin,MiT-B0,Conv Drop,0.635
97,SplitLawin,MiT-B1,Conv Drop,0.630
6,DoubleLawin,MiT-B0,SE,0.534
18,DoubleLawin,MiT-B1,SE,0.557


In [64]:
filters = {
    "$and": [
    {"$or":[
        {'config.in_params.model.name': 'splitlawin'},
        {'config.in_params.model.name': 'doublelawin'}, 
    ]},
    {'config.in_params.train_params.loss.name': 'focal'}
    ]
}
selected_cols = [
    'in_params.model.name',
    'in_params.model.params.backbone',
    'in_params.model.params.fusion_type',
    'f1',
]
roundings = ['F1']
runs_df = get_runs(filters, project="pasqualedem/weedmapping-rededge")

In [65]:
projected_runs = runs_df[
    selected_cols
]
selected_runs = projected_runs
selected_runs.rename(columns=renamings, inplace=True)
rename_values(selected_runs, value_renamings)
selected_runs[roundings] = selected_runs[roundings].round(5)
selected_runs.fillna(0, inplace=True)
selected_runs = selected_runs.applymap(lambda x: ', '.join(x) if isinstance(x, list) else x)
selected_runs.sort_values(by=['Fusion', 'Model', 'Backbone'], inplace=True)
idx = selected_runs.groupby(by=['Model', 'Backbone', 'Fusion'])['F1'].transform(max) == selected_runs['F1']
best_runs = selected_runs[idx]
best_runs[roundings] = best_runs[roundings].round(3)
selected_runs[roundings] = selected_runs[roundings].round(3)

/tmp/ipykernel_441543/1509668528.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_runs.rename(columns=renamings, inplace=True)
/tmp/ipykernel_441543/2946659532.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column] = df[column].replace(values)
/tmp/ipykernel_441543/1509668528.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returnin

In [66]:
best_runs

,Model,Backbone,Fusion,F1
31,DoubleLawin,MiT-B0,Conv,0.863
24,DoubleLawin,MiT-B1,Conv,0.862
109,SplitLawin,MiT-B0,Conv,0.859
101,SplitLawin,MiT-B1,Conv,0.865
27,DoubleLawin,MiT-B0,Conv Drop,0.863
20,DoubleLawin,MiT-B1,Conv Drop,0.863
108,SplitLawin,MiT-B0,Conv Drop,0.862
98,SplitLawin,MiT-B1,Conv Drop,0.862
8,DoubleLawin,MiT-B0,SE,0.857
0,DoubleLawin,MiT-B1,SE,0.857
